# Face Recognition and Verification in an Image

## Required Libraries

In [5]:
import face_recognition
import pickle
import cv2
import imutils

## About the Code 

```
with open("face-labels-single.pickle", 'rb') as f:
```
First we load in the dictionary with the list of encodings and labels for verification

Next we read the image we want to recognize into a variable and resize it to 480px(for faster processing). Then, we convert it
to an RGB format.

After that we retrieve the faces in the form of coordinates using ```face_recognition.face_locations()``` and since we only have to process 1 image we can afford to use cnn as our model for face detection.
The ```number_of_times_to_upsample``` parameter(optional) essentially tells the code to look for smaller faces(default is 1). Would recommend to let it be 1 as it often leads to false positives when you're trying to detect smaller faces. 


NOW, once we have all the encodings i.e all the faces detected we try and and match them with the known faces encodings

### How we do this you ask?

We compare the encoding from the image with known ones by calculating the <b>Euclidian Distance</b>

Euclidean distance is the length of a straight line between two vectors in Euclidean space. i.e. 

<img src="http://mathworld.wolfram.com/images/equations/Distance/NumberedEquation3.gif">

Once we compare them we classify them as match or not for all of the encodings through the following code

```matches = face_recognition.compare_faces(data["encodings"],encoding,tolerance=0.45)```

        
The ``` tolerance ``` acts as the threshold for comparison , if the Euclidiean distance is less than 0.45 then the encoding is classified as a match(True) else not a match(fals)

Hence matches is a grid of size (known faces * encodings found in the image). Next we parse through the grid and find out which encoding has the maximum number of matches and classify that as our found face.

Finally we take all the faces verfied and print them on the image.

#### That's it folks! We have a Face Verification Program


In [7]:
with open("face-labels-single.pickle", 'rb') as f:
    data = pickle.load(f)

# load the input image and convert it from BGR to RGB
image = cv2.imread("C:/Users/Vansh/Desktop/b.jpg")
image = imutils.resize(image, width=480)
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# detect the (x, y)-coordinates of the bounding boxes corresponding
# to each face in the input image, then compute the facial embeddings
# for each face
boxes = face_recognition.face_locations(rgb,number_of_times_to_upsample=1,model='cnn')
encodings = face_recognition.face_encodings(rgb, boxes)

# initialize the list of names for each face detected
names = []

# loop over the facial embeddings
for encoding in encodings:
    # attempt to match each face in the input image to our known
    # encodings
    matches = face_recognition.compare_faces(data["encodings"],
        encoding,tolerance=0.45)
    name = "Unknown"

    # check to see if we have found a match
    if True in matches:
        # find the indexes of all matched faces then initialize a
        # dictionary to count the total number of times each face
        # was matched
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}

        # loop over the matched indexes and maintain a count for
        # each recognized face face
        for i in matchedIdxs:
            name = data["names"][i]
            counts[name] = counts.get(name, 0) + 1

        # determine the recognized face with the largest number of
        # votes (note: in the event of an unlikely tie Python will
        # select first entry in the dictionary)
        name = max(counts, key=counts.get)

    # update the list of names
    names.append(name)

# loop over the recognized faces
for ((top, right, bottom, left), name) in zip(boxes, names):
    # draw the predicted face name on the image
    cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
    y = top - 15 if top - 15 > 15 else top + 15
    cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

# show the output image
cv2.imshow("Image", image)
cv2.waitKey(0)

-1